<a href="https://colab.research.google.com/github/bhuvansun/rice-classification/blob/main/02_insurance_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [ ]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [ ]:
!pip install jovian

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6478 sha256=32a09f19acbccdd6cff7f42ff49c7e31b8807754b8e9bc7a85310acf1cc843f9
  Stored in directory: /root/.cache/pip/wheels/ed/08/9e/f0a977dfe55051a07e21af89200125d65f1efa60cbac61ed88
Successfully built uuid


In [ ]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file.

In [ ]:
DATASET_URL = "https://gist.github.com/BirajCoder/5f068dfe759c1ea6bdfce9535acdb72d/raw/c84d84e3c80f93be67f6c069cbdc0195ec36acbd/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

100%|██████████| 54288/54288 [00:00<00:00, 19048048.82it/s]


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [ ]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [ ]:
your_name = 'Bhuvan' # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [ ]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [ ]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,region,charges
397,21,male,32.2608,0,no,southeast,19406.202321
211,40,male,32.1100,4,no,northwest,9550.378012
304,57,female,39.5200,2,no,southwest,14796.062190
349,19,male,28.9484,0,no,northwest,1913.808370
733,48,female,28.3556,1,no,northeast,11053.282909


Let us answer some basic questions about the dataset.


**Q1: How many rows does the dataset have?**

In [ ]:
num_rows = len(dataframe)
print(num_rows)

1271


**Q2: How many columns doe the dataset have**

In [ ]:
num_cols = len(dataframe.columns)
print(num_cols)

7


**Q3: What are the column titles of the input variables?**

In [ ]:
input_cols = ['age', 'sex', 'bmi', 'children', 'smoker', 'region']

**Q4: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [ ]:
categorical_cols = ['sex', 'smoker', 'region']

**Q5: What are the column titles of output/target variable(s)?**

In [ ]:
output_cols = ['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ai/aakashns/dataviz-cheatsheet

In [ ]:
# Write your answer here
dataframe.charges.min(), dataframe.charges.max(), dataframe.charges.mean()

(1312.592463, 74611.4007717, 15502.293266117256)

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [ ]:
!pip install jovian --upgrade -q

In [ ]:
import jovian

In [ ]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.


## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [ ]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [ ]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[21.    ,  1.    , 32.2608,  0.    ,  0.    ,  2.    ],
        [40.    ,  1.    , 32.11  ,  4.    ,  0.    ,  1.    ],
        [57.    ,  0.    , 39.52  ,  2.    ,  0.    ,  3.    ],
        ...,
        [19.    ,  0.    , 26.7748,  1.    ,  0.    ,  1.    ],
        [62.    ,  0.    , 26.    ,  0.    ,  0.    ,  3.    ],
        [46.    ,  1.    , 45.6456,  3.    ,  0.    ,  2.    ]]),
 array([[19406.2023207],
        [ 9550.3780125],
        [14796.06219  ],
        ...,
        [ 3171.6694035],
        [15737.81274  ],
        [10464.614667 ]]))

**Q6: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [ ]:
inputs = torch.from_numpy(inputs_array).to(torch.float32)
targets = torch.from_numpy(targets_array).to(torch.float32)

In [ ]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [ ]:
dataset = TensorDataset(inputs, targets)

**Q7: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [ ]:
val_percent = 0.1 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = torch.utils.data.random_split(dataset, [val_size, train_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q8: Pick a batch size for the data loader.**

In [ ]:
batch_size = 50

In [ ]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [ ]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[36.0000,  1.0000, 29.1460,  1.0000,  1.0000,  0.0000],
        [19.0000,  1.0000, 36.1920,  0.0000,  1.0000,  3.0000],
        [23.0000,  1.0000, 35.7760,  0.0000,  0.0000,  3.0000],
        [37.0000,  0.0000, 30.6800,  2.0000,  0.0000,  3.0000],
        [38.0000,  0.0000, 29.1200,  3.0000,  0.0000,  3.0000],
        [26.0000,  1.0000, 33.7896,  1.0000,  0.0000,  0.0000],
        [19.0000,  1.0000, 21.2420,  0.0000,  0.0000,  1.0000],
        [33.0000,  0.0000, 41.4128,  1.0000,  0.0000,  2.0000],
        [55.0000,  0.0000, 31.0232,  0.0000,  0.0000,  0.0000],
        [41.0000,  0.0000, 29.4424,  1.0000,  0.0000,  1.0000],
        [25.0000,  1.0000, 37.0500,  0.0000,  0.0000,  1.0000],
        [44.0000,  1.0000, 28.4960,  2.0000,  0.0000,  3.0000],
        [28.0000,  1.0000, 28.0592,  2.0000,  0.0000,  0.0000],
        [34.0000,  0.0000, 28.6000,  1.0000,  0.0000,  3.0000],
        [51.0000,  1.0000, 25.7868,  2.0000,  1.0000,  1.0000],
        [32.0000,  0.0000, 45.98

Let's save our work by committing to Jovian.

In [ ]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.


## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment).


In [ ]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q9: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [ ]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                  # fill this (hint: use input_size & output_size defined above)

    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out

    def training_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calcuate loss
        loss = torch.nn.functional.l1_loss(out, targets)                          # fill this
        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = torch.nn.functional.l1_loss(out, targets)                           # fill this
        return {'val_loss': loss.detach()}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [ ]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2790,  0.1074, -0.3337, -0.2828, -0.1162, -0.1332]],
        requires_grad=True),
 Parameter containing:
 tensor([0.2908], requires_grad=True)]

One final commit before we train the model.

In [ ]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.


## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [ ]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q10: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [ ]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 15699.4208984375}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q11: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [ ]:
epochs = 1000
lr = 1e-2
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 14206.0664
Epoch [40], val_loss: 12791.6250
Epoch [60], val_loss: 11649.6768
Epoch [80], val_loss: 10765.6543
Epoch [100], val_loss: 10110.2715
Epoch [120], val_loss: 9611.0771
Epoch [140], val_loss: 9258.5000
Epoch [160], val_loss: 9033.7295
Epoch [180], val_loss: 8897.4922
Epoch [200], val_loss: 8812.6182
Epoch [220], val_loss: 8762.5039
Epoch [240], val_loss: 8733.4990
Epoch [260], val_loss: 8715.8672
Epoch [280], val_loss: 8701.9746
Epoch [300], val_loss: 8690.4297
Epoch [320], val_loss: 8680.6084
Epoch [340], val_loss: 8669.2266
Epoch [360], val_loss: 8658.0498
Epoch [380], val_loss: 8647.0283
Epoch [400], val_loss: 8638.1094
Epoch [420], val_loss: 8629.5059
Epoch [440], val_loss: 8618.1865
Epoch [460], val_loss: 8610.4775
Epoch [480], val_loss: 8598.9863
Epoch [500], val_loss: 8589.2393
Epoch [520], val_loss: 8580.4795
Epoch [540], val_loss: 8570.2812
Epoch [560], val_loss: 8559.7988
Epoch [580], val_loss: 8548.0371
Epoch [600], val_loss: 8538.4053
Epoch [62

In [ ]:
epochs = 2000
lr = 1e-1
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 8291.6016
Epoch [40], val_loss: 8233.2754
Epoch [60], val_loss: 8193.0303
Epoch [80], val_loss: 8145.7983
Epoch [100], val_loss: 8126.3247
Epoch [120], val_loss: 8111.4448
Epoch [140], val_loss: 8101.8213
Epoch [160], val_loss: 8104.8564
Epoch [180], val_loss: 8098.0278
Epoch [200], val_loss: 8090.8926
Epoch [220], val_loss: 8096.1719
Epoch [240], val_loss: 8090.3613
Epoch [260], val_loss: 8090.2549
Epoch [280], val_loss: 8089.4082
Epoch [300], val_loss: 8099.9604
Epoch [320], val_loss: 8085.9580
Epoch [340], val_loss: 8092.7432
Epoch [360], val_loss: 8089.1357
Epoch [380], val_loss: 8090.7134
Epoch [400], val_loss: 8094.6045
Epoch [420], val_loss: 8087.6704
Epoch [440], val_loss: 8077.7881
Epoch [460], val_loss: 8087.7202
Epoch [480], val_loss: 8078.2812
Epoch [500], val_loss: 8092.1904
Epoch [520], val_loss: 8078.0127
Epoch [540], val_loss: 8074.6055
Epoch [560], val_loss: 8089.3042
Epoch [580], val_loss: 8081.7124
Epoch [600], val_loss: 8079.8823
Epoch [620], v

In [ ]:
epochs = 3000
lr = 1e-1
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 8043.7192
Epoch [40], val_loss: 8035.1030
Epoch [60], val_loss: 8030.3335
Epoch [80], val_loss: 8041.4883
Epoch [100], val_loss: 8031.9932
Epoch [120], val_loss: 8033.0083
Epoch [140], val_loss: 8037.8887
Epoch [160], val_loss: 8039.7134
Epoch [180], val_loss: 8039.1318
Epoch [200], val_loss: 8032.6108
Epoch [220], val_loss: 8032.4873
Epoch [240], val_loss: 8027.1997
Epoch [260], val_loss: 8040.3887
Epoch [280], val_loss: 8038.3330
Epoch [300], val_loss: 8030.9536
Epoch [320], val_loss: 8037.8735
Epoch [340], val_loss: 8033.1582
Epoch [360], val_loss: 8033.6235
Epoch [380], val_loss: 8039.3218
Epoch [400], val_loss: 8032.2437
Epoch [420], val_loss: 8029.6680
Epoch [440], val_loss: 8027.3145
Epoch [460], val_loss: 8030.2905
Epoch [480], val_loss: 8026.4062
Epoch [500], val_loss: 8022.0107
Epoch [520], val_loss: 8021.5796
Epoch [540], val_loss: 8019.5532
Epoch [560], val_loss: 8032.3643
Epoch [580], val_loss: 8020.0508
Epoch [600], val_loss: 8023.1094
Epoch [620], v

In [ ]:
epochs = 500
lr = 1e-1
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 7975.3438
Epoch [40], val_loss: 7973.9551
Epoch [60], val_loss: 7961.3022
Epoch [80], val_loss: 7958.1724
Epoch [100], val_loss: 7956.1914
Epoch [120], val_loss: 7961.0708
Epoch [140], val_loss: 7967.0332
Epoch [160], val_loss: 7964.5767
Epoch [180], val_loss: 7975.5698
Epoch [200], val_loss: 7959.6562
Epoch [220], val_loss: 7958.9126
Epoch [240], val_loss: 7966.8179
Epoch [260], val_loss: 7962.3018
Epoch [280], val_loss: 7962.9062
Epoch [300], val_loss: 7956.5508
Epoch [320], val_loss: 7959.4702
Epoch [340], val_loss: 7958.0107
Epoch [360], val_loss: 7953.5342
Epoch [380], val_loss: 7962.0742
Epoch [400], val_loss: 7956.8838
Epoch [420], val_loss: 7956.1040
Epoch [440], val_loss: 7955.9790
Epoch [460], val_loss: 7961.8613
Epoch [480], val_loss: 7959.5747
Epoch [500], val_loss: 7958.7324


In [ ]:
epochs = 10000
lr = 1e-1
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 7962.1343
Epoch [40], val_loss: 7957.8730
Epoch [60], val_loss: 7959.5220
Epoch [80], val_loss: 7951.8804
Epoch [100], val_loss: 7951.4775
Epoch [120], val_loss: 7949.1509
Epoch [140], val_loss: 7963.3589
Epoch [160], val_loss: 7948.2017
Epoch [180], val_loss: 7952.0239
Epoch [200], val_loss: 7955.6196
Epoch [220], val_loss: 7951.4360
Epoch [240], val_loss: 7948.7969
Epoch [260], val_loss: 7964.2773
Epoch [280], val_loss: 7956.7148
Epoch [300], val_loss: 7951.3296
Epoch [320], val_loss: 7946.8843
Epoch [340], val_loss: 7949.5571
Epoch [360], val_loss: 7957.8501
Epoch [380], val_loss: 7942.9429
Epoch [400], val_loss: 7941.9502
Epoch [420], val_loss: 7945.6089
Epoch [440], val_loss: 7946.7427
Epoch [460], val_loss: 7947.0659
Epoch [480], val_loss: 7949.5645
Epoch [500], val_loss: 7938.4985
Epoch [520], val_loss: 7949.5000
Epoch [540], val_loss: 7942.0454
Epoch [560], val_loss: 7940.3096
Epoch [580], val_loss: 7951.4482
Epoch [600], val_loss: 7943.3345
Epoch [620], v

**Q12: What is the final validation loss of your model?**

In [ ]:
val_loss = 7789.0630

Let's log the final validation loss to Jovian and commit the notebook

In [ ]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Please enter your API key ( from https://jovian.com/ ):
API KEY: ··········
[jovian] Metrics logged.


In [ ]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.


Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q13: Complete the following function definition to make predictions on a single input**

In [ ]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(input)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [ ]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([19.0000,  1.0000, 26.1820,  0.0000,  0.0000,  1.0000])
Target: tensor([1909.4824])
Prediction: tensor(2777.6504)


In [ ]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([41.0000,  0.0000, 32.8640,  0.0000,  0.0000,  3.0000])
Target: tensor([7237.7686])
Prediction: tensor(8144.0010)


In [ ]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([53.0000,  1.0000, 30.6592,  0.0000,  0.0000,  2.0000])
Target: tensor([11100.5439])
Prediction: tensor(11964.8955)


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog.

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [ ]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.
[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.
